In [3]:
from modelsandbox.model.containers import Container
from modelsandbox.model.processors import Function, LogicSchema

import math

class CPM(Container):
    """
    Crash prediction model for rural two-lane roads based on HSM chapter 10.
    """

    class SPFs(Container, order=0):
        """
        Safety performance functions for rural two-lane roads.
        """

        test_schema = LogicSchema({
            "label": "test_schema",
            "params": ["lookup"],
            "actions": ["lte"],
            "data": {
                0: {"a": 1, "b": 2},
                1: {"a": 3, "b": 4},
                2: {"a": 5, "b": 6},
                999: {"a": 7, "b": 8}
            }
        })

        @Function(order=0).wrapper
        def test(a, b):
            return a + b

        @Function(order=0).wrapper
        def n_kabco(aadt, length):
            """
            Based on HSM Equation 10-7.
            """
            # Compute number of crashes
            n = aadt * length * 365 * 10E-6 * math.exp(-0.312)
            return n

        @Function(order=0).wrapper
        def overdispersion(length):
            # Compute overdispersion
            k = 0.236 / length
            return k

    class AFs(Container, order=1):
        """
        Adjustment factors for rural two-lane roads.
        """
        
        @Function(hidden=True).wrapper
        def af_shoulder_width(shoulder_width):
            return 1 / (shoulder_width / 8)
        
        @Function(hidden=True).wrapper
        def af_lane_width(lane_width):
            return 1 / (lane_width / 12)
        
        @Function(order=99).wrapper
        def af_total(af_shoulder_width, af_lane_width):
            return af_shoulder_width * af_lane_width
        
    class Predictions(Container, order=2):
        """
        Predictions for rural two-lane roads.
        """
        
        @Function(order=0).wrapper
        def predict_crashes(n_kabco, af_total):
            # Compute predicted crashes
            return n_kabco * af_total
        
cpm = CPM()

In [4]:
cpm.processors[0][3].label

In [ ]:
cpm = CPM()
cpm.analyze(a=1, b=2, aadt=1000, length=10, shoulder_width=6, lane_width=11.5)

In [ ]:
class Test:

    x = 1
    y = 2
    Function(lambda z: z + 1)

t = Test()
import inspect
[m for m in inspect.getmembers(t) if m[0][:2] != "__"]